# **PiDL**

In [12]:
import numpy as np

def STRidge(U9, Phi, delta, max_iter=1000, lambda_init=None, tol=1e-6, ridge_reg_param=1e-5, dynamic_ridge=True):
    """
    Enhanced Sequential Threshold Ridge Regression (STRidge)
    
    Args:
        U9 (numpy.ndarray): Time derivative vector (target vector).
        Phi (numpy.ndarray): Candidate function library matrix.
        delta (float): Threshold tolerance for sparsity.
        max_iter (int): Maximum number of iterations.
        lambda_init (numpy.ndarray, optional): Initial coefficients for λ. Default is zeros.
        tol (float): Convergence tolerance for coefficients.
        ridge_reg_param (float): Initial ridge regularization parameter.
        dynamic_ridge (bool): Whether to decrease ridge_reg_param over iterations.
    
    Returns:
        numpy.ndarray: The final coefficients λ.
    """
    # Initialize coefficients λ
    if lambda_init is None:
        lambda_ = np.zeros(Phi.shape[1])
    else:
        lambda_ = lambda_init
    
    # Iterative updates
    for iteration in range(max_iter):
        # Store previous coefficients for convergence check
        prev_lambda = lambda_.copy()
        
        # Step 4: Identify small (I) and large (J) coefficients
        I = np.where(np.abs(lambda_) < delta)[0]  # Small coefficients
        J = np.where(np.abs(lambda_) >= delta)[0]  # Large coefficients
        
        # Step 5: Enforce sparsity by zeroing small coefficients
        lambda_[I] = 0
        
        # Step 6: Ridge regression update for large coefficients (J)
        if len(J) > 0:
            Phi_J = Phi[:, J]  # Submatrix corresponding to non-zero coefficients
            A = Phi_J.T @ Phi_J + ridge_reg_param * np.eye(len(J))  # Regularized Gram matrix
            b = Phi_J.T @ U9  # Target vector for regression
            
            # Solve the ridge regression problem
            lambda_[J] = np.linalg.solve(A, b)
        
        # Debugging logs
        print(f"Iteration {iteration + 1}: Non-zero coefficients = {len(J)}")
        
        # Convergence check
        if np.linalg.norm(lambda_ - prev_lambda, ord=2) < tol:
            print("Converged!")
            break
        
        # Optionally decrease the ridge regularization parameter
        if dynamic_ridge:
            ridge_reg_param *= 0.9  # Reduce by 10% each iteration

    return lambda_


In [11]:
import numpy as np

def generate_collocation_points(x_range, t_range, n_points, method="uniform"):
    """
    Generate collocation points in the spatio-temporal domain.
    
    Args:
        x_range (tuple): (x_min, x_max), spatial domain.
        t_range (tuple): (t_min, t_max), temporal domain.
        n_points (int): Total number of collocation points.
        method (str): Sampling method - "uniform", "random", "sobol".
        
    Returns:
        numpy.ndarray: Array of collocation points [N_c, 2], where each row is (x_i, t_i).
    """
    x_min, x_max = x_range
    t_min, t_max = t_range

    if method == "uniform":
        # Create a uniform grid of points
        n_x = int(np.sqrt(n_points))  # Number of points in x
        n_t = int(np.sqrt(n_points))  # Number of points in t
        x = np.linspace(x_min, x_max, n_x)
        t = np.linspace(t_min, t_max, n_t)
        x, t = np.meshgrid(x, t)
        points = np.column_stack([x.ravel(), t.ravel()])

    elif method == "random":
        # Randomly sample points
        x = np.random.uniform(x_min, x_max, n_points)
        t = np.random.uniform(t_min, t_max, n_points)
        points = np.column_stack([x, t])

    elif method == "sobol":
        # Use Sobol sequence for quasi-random sampling
        from scipy.stats.qmc import Sobol
        sobol = Sobol(d=2)
        samples = sobol.random(n_points)
        x = x_min + (x_max - x_min) * samples[:, 0]
        t = t_min + (t_max - t_min) * samples[:, 1]
        points = np.column_stack([x, t])

    else:
        raise ValueError("Unsupported method. Choose from 'uniform', 'random', 'sobol'.")
    
    return points

# Example usage:
x_range = (0, 1)  # Spatial domain
t_range = (0, 1)  # Temporal domain
n_points = 1000   # Total number of collocation points

# Generate collocation points using uniform grid
collocation_points = generate_collocation_points(x_range, t_range, n_points, method="uniform")
print("Collocation Points:\n", collocation_points)


Collocation Points:
 [[0.         0.        ]
 [0.03333333 0.        ]
 [0.06666667 0.        ]
 ...
 [0.93333333 1.        ]
 [0.96666667 1.        ]
 [1.         1.        ]]


In [3]:
import numpy as np
from itertools import combinations_with_replacement

def generate_candidate_functions(variables, d_max, trigonometric=True, additional_functions=['log', 'exp']):
    terms = []
    
    # 1. Constant term
    terms.append(1)
    
    # 2. Polynomial terms
    for var in variables:
        for d in range(1, d_max + 1):
            terms.append(var**d)
    
    # 3. Mixed polynomial terms
    for d in range(2, d_max + 1):
        for comb in combinations_with_replacement(variables, d):
            term = 1
            for v in comb:
                term *= v
            terms.append(term)
    
    # 4. Trigonometric terms
    if trigonometric:
        for var in variables:
            terms.append(np.sin(var))
            terms.append(np.cos(var))
    
    # 5. Mixed trigonometric terms
    if trigonometric:
        for comb in combinations_with_replacement(variables, 2):
            terms.append(np.sin(comb[0]) * np.cos(comb[1]))
    
    # 6. Additional function terms: Logarithmic and Exponential
    for var in variables:
        if 'log' in additional_functions:
            terms.append(np.log(var))
        if 'exp' in additional_functions:
            terms.append(np.exp(var))
    
    # 7. Mixed Additional Function Terms
    for comb in combinations_with_replacement(variables, 2):
        if 'log' in additional_functions and 'exp' in additional_functions:
            terms.append(np.log(comb[0]) * np.exp(comb[1]))
    
    return np.array(terms)

# Example Usage:
variables = ['x', 't']
d_max = 3  # Allowing higher-degree polynomials up to x^3
candidate_matrix = generate_candidate_functions(variables, d_max)
print(candidate_matrix)


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [13]:
import torch

def generate_candidate_library(model, x, t, max_degree=2):
    """
    Generate candidate terms automatically using a neural network model in PyTorch.
    
    Args:
        model: Neural network that predicts u(x, t).
        x: Tensor of spatial points (requires gradient).
        t: Tensor of temporal points (requires gradient).
        max_degree: Maximum polynomial degree to include.
        
    Returns:
        candidates (dict): Dictionary of candidate terms evaluated at (x, t).
        Phi (Tensor): Candidate library matrix (N_collocation_points x N_terms).
    """
    # Ensure inputs require gradients for automatic differentiation
    x.requires_grad_(True)
    t.requires_grad_(True)

    # Compute the prediction and first-order derivatives
    u = model(x, t)  # Neural network output
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    # Compute second-order derivatives
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

    # Construct basic terms
    candidates = {
        "1": torch.ones_like(u),  # Constant term
        "u": u,                  # Predicted field variable
        "u_x": u_x,              # First spatial derivative
        "u_t": u_t,              # First temporal derivative
        "u_xx": u_xx             # Second spatial derivative
    }

    # Add polynomial terms (e.g., u^2, u^3, ...)
    for i in range(2, max_degree + 1):
        candidates[f"u^{i}"] = u ** i

    # Add mixed terms (e.g., u * u_x, u * u_t)
    candidates["u * u_x"] = u * u_x
    candidates["u * u_t"] = u * u_t

    # Combine terms into a candidate library matrix (Φ)
    Phi = torch.cat([term.reshape(-1, 1) for term in candidates.values()], dim=1)

    return candidates, Phi


In [5]:
import torch.nn as nn

class PhysicsInformedNN(nn.Module):
    def __init__(self):
        super(PhysicsInformedNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 50),   # Input: (x, t)
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 1)    # Output: u(x, t)
        )

    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=1)  # Concatenate x and t along columns
        return self.model(inputs)


In [6]:
import numpy as np

def generate_collocation_points(x_range, t_range, n_points):
    """
    Generate collocation points in the spatio-temporal domain.
    
    Args:
        x_range: Tuple specifying the spatial range (x_min, x_max).
        t_range: Tuple specifying the temporal range (t_min, t_max).
        n_points: Number of collocation points.
        
    Returns:
        x, t: Tensors of spatial and temporal points.
    """
    x = np.random.uniform(x_range[0], x_range[1], n_points)
    t = np.random.uniform(t_range[0], t_range[1], n_points)
    x = torch.tensor(x, dtype=torch.float32).reshape(-1, 1)
    t = torch.tensor(t, dtype=torch.float32).reshape(-1, 1)
    return x, t

# Example: Generate 1000 collocation points in [0, 1] x [0, 1]
x_range, t_range = (0, 1), (0, 1)
x, t = generate_collocation_points(x_range, t_range, n_points=1000)


In [7]:
# Instantiate the neural network
model = PhysicsInformedNN()

# Generate the candidate terms library
max_degree = 2  # Include terms up to u^2
candidates, Phi = generate_candidate_library(model, x, t, max_degree)

# Print the candidate terms
print("Candidate Terms:")
for term_name, values in candidates.items():
    print(f"{term_name}: Shape = {values.shape}")

print("\nCandidate Library (Φ):")
print(Phi.shape)  # Output: (1000, Number of candidate terms)


Candidate Terms:
1: Shape = torch.Size([1000, 1])
u: Shape = torch.Size([1000, 1])
u_x: Shape = torch.Size([1000, 1])
u_t: Shape = torch.Size([1000, 1])
u_xx: Shape = torch.Size([1000, 1])
u^2: Shape = torch.Size([1000, 1])
u * u_x: Shape = torch.Size([1000, 1])
u * u_t: Shape = torch.Size([1000, 1])

Candidate Library (Φ):
torch.Size([1000, 8])


In [8]:
Phi

tensor([[ 1.0000, -0.0684,  0.0452,  ...,  0.0047, -0.0031,  0.0102],
        [ 1.0000, -0.1540,  0.0972,  ...,  0.0237, -0.0150,  0.0153],
        [ 1.0000, -0.1439,  0.0890,  ...,  0.0207, -0.0128,  0.0158],
        ...,
        [ 1.0000, -0.0398,  0.0305,  ...,  0.0016, -0.0012,  0.0067],
        [ 1.0000, -0.1452,  0.0526,  ...,  0.0211, -0.0076,  0.0235],
        [ 1.0000, -0.0679,  0.0433,  ...,  0.0046, -0.0029,  0.0114]],
       grad_fn=<CatBackward0>)

In [9]:
def normalize_columns(Phi):
    """
    Normalize each column of the candidate library Φ.
    """
    norms = torch.norm(Phi, dim=0, keepdim=True)  # Compute column norms
    Phi_normalized = Phi / norms
    return Phi_normalized

# Normalize Φ
Phi_normalized = normalize_columns(Phi)


In [10]:
Phi_normalized

tensor([[ 0.0316, -0.0177,  0.0224,  ...,  0.0081, -0.0113,  0.0195],
        [ 0.0316, -0.0398,  0.0481,  ...,  0.0413, -0.0545,  0.0292],
        [ 0.0316, -0.0372,  0.0440,  ...,  0.0361, -0.0466,  0.0300],
        ...,
        [ 0.0316, -0.0103,  0.0151,  ...,  0.0028, -0.0044,  0.0127],
        [ 0.0316, -0.0375,  0.0260,  ...,  0.0367, -0.0278,  0.0448],
        [ 0.0316, -0.0176,  0.0214,  ...,  0.0080, -0.0107,  0.0216]],
       grad_fn=<DivBackward0>)